In [1]:
from datasets import load_dataset

In [2]:
rawdata = load_dataset("csv", data_files={'train': ["../data/corpus_train.csv"], 
                                       'validation': ["../data/corpus_valid.csv"], 
                                       "test": ["../data/corpus_test.csv"]})

Using custom data configuration default-a7ffed3cafb72707
Reusing dataset csv (/Users/ravenwcy/.cache/huggingface/datasets/csv/default-a7ffed3cafb72707/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# load the plm
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")

/opt/anaconda3/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [4]:
# construct a template
from openprompt.prompts import ManualTemplate
template_text = '{"placeholder":"text_a"} In this sentence, the topic is {"mask"}.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

In [14]:
rawdata["train"]["label"]

['assumption',
 'assumption',
 'assumption',
 'testimony',
 'anecdote',
 'assumption',
 'assumption',
 'anecdote',
 'anecdote',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'anecdote',
 'assumption',
 'statistics',
 'assumption',
 'anecdote',
 'title',
 'testimony',
 'assumption',
 'testimony',
 'assumption',
 'assumption',
 'assumption',
 'statistics',
 'assumption',
 'anecdote',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'anecdote',
 'assumption',
 'anecdote',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'anecdote',
 'assumption',
 'assumption',
 'statistics',
 'assumption',
 'testimony',
 'assumption',
 'assumption',
 'assumption',
 'anecdote',
 'anecdote',
 'testimony',
 'assumption',
 'assumption',
 'assumption',
 'assumption',
 'title',
 'assumption',
 'assumption',
 'assumption',
 'statistics',
 '

In [33]:
def int_label(label):
    if label == "assumption": return 0
    elif label == "testimony": return 1
    elif label == "anecdote": return 2
    elif label == "statistics": return 3
    elif label == "title": return 4
    elif label == "common-ground": return 5
    elif label == "other": return 6

In [34]:
# convert our raw data to openprompt's form
from openprompt.data_utils import InputExample
dataset = {}
for split in ['train', 'validation', 'test']:
    dataset[split] = []
    for data in rawdata[split]:
        input_example = InputExample(text_a = data['sentence'], label = int_label(data['label']), guid=data['article_id'])
        dataset[split].append(input_example)
print(dataset['train'][0])

{
  "guid": 45,
  "label": 0,
  "meta": {},
  "text_a": "which are obviously bad enough",
  "text_b": "",
  "tgt_text": null
}



In [25]:
# wrapped_example = mytemplate.wrap_one_example(dataset["train"][0])
# print(wrapped_example)

[[{'text': 'which are obviously bad enough', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' In this sentence, the topic is', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': '.', 'loss_ids': 0, 'shortenable_ids': 0}], {'guid': 45, 'label': 'assumption'}]


In [26]:
# # T5: pass <pad> <extra_id_0> <eos> to decoder
# wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, 
#                                    tokenizer=tokenizer,truncate_method="head")

In [19]:
# # Tokenize example
# tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
# print(tokenized_example)
# print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
# print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))

In [28]:
# tokenize for whole dataset
#model_inputs = {}
#for split in ['train', 'validation', 'test']:
    #model_inputs[split] = []
    #for sample in dataset[split]:
        #tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        #model_inputs[split].append(tokenized_example)

In [35]:
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 9505it [00:13, 685.48it/s]


In [36]:
from openprompt.prompts import ManualVerbalizer
import torch

In [37]:
# for example the verbalizer contains multiple label words in each class
myverbalizer = ManualVerbalizer(tokenizer, num_classes=7,
                        label_words=[["assumption"], ["testimony"], ["anecdote"], ["statistics"],["title"],["common-ground"],["other"]])

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do

Parameter containing:
tensor([[[20662,     0,     0,     0,     0,     0,     0]],

        [[16956,     0,     0,     0,     0,     0,     0]],

        [[   46,    15,    75,    26,    32,    17,    15]],

        [[ 7475,     0,     0,     0,     0,     0,     0]],

        [[ 2233,     0,     0,     0,     0,     0,     0]],

        [[ 1017,    18,  9232,     0,     0,     0,     0]],

        [[  119,     0,     0,     0,     0,     0,     0]]])
tensor([[-1.2388, -2.6150, -2.6378, -2.5801, -2.8246, -1.1137, -2.2813],
        [-2.1162, -2.3493, -2.1118, -1.1165, -3.1388, -2.3782, -1.6115]])


In [38]:
from openprompt import PromptForClassification
#use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
#if use_cuda: prompt_model=  prompt_model.cuda()

In [39]:
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [40]:
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

/opt/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
for epoch in range(2):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        #if use_cuda:
            #inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %100 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)


Epoch 0, average loss: 3.2245185375213623
Epoch 0, average loss: 0.954590522848508


In [ ]:
# Evaluate
validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)